# PageRank

In [2]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

In [ ]:
# gds.run_cypher("""
#                CALL gds.graph.project(
#                    'popularQuestionAlternatives',
#                    ['QuestionAlternative', 'Respondent'],
#                    {
#                         CHOSE: {
#                             orientation: 'REVERSE'
#                         }
#                    }
#                )
#                """)

In [ ]:
# gds.run_cypher("""
#                CALL gds.degree.write.estimate('popularQuestionAlternatives', 
#                { writeProperty: 'degree' })
#                YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
#                """)

In [ ]:
# gds.run_cypher("""
#                CALL gds.graph.relationshipProperties.write('popularQuestionAlternatives', {
#                      relationshipProperties: {
#                             CHOSE: {
#                                 property: 'score',
#                                 defaultValue: 1.0
#                             }
#                         }
#                     })
#                """)

In [ ]:
gds.run_cypher("""
               CALL gds.degree.mutate('popularQuestionAlternatives',
                { mutate })
                YIELD nodeId, score
                WHERE score > 0
                RETURN gds.util.asNode(nodeId).name AS name, score AS respondents
                ORDER BY respondents ASC, name ASC
               """)

In [ ]:
gds.run_cypher("""
               CALL gds.degree.stats('popularQuestionAlternatives')
               YIELD centralityDistribution
               RETURN centralityDistribution.min as minimumScore, centralityDistribution.max as maximumScore, centralityDistribution.mean as meanScore
               """)


In [ ]:
gds.run_cypher("""
               CALL gds.degree.stream(
                    'popularQuestionAlternatives'
               )
               YIELD nodeId, score
               RETURN gds.util.asNode(nodeId).name AS name, score AS respondents
               ORDER by respondents DESC, name ASC
               """)

In [ ]:
gds.run_cypher("""
               CALL gds.degree.write(
                    'popularQuestionAlternatives',
                    { writeProperty: 'nrOfRespondents' }
               )
               YIELD nodePropertiesWritten
               RETURN nodePropertiesWritten
               """)

In [ ]:
# Create relationship from questionalternative to respondent called CHOSEN_BY if the there already is a relationship between the two nodes called CHOSE. 
# The relationship CHOSEN_BY will have a property called score which is the same as the property nrOfRespondents on the node QuestionAlternative.
gds.run_cypher("""
                MATCH (q:QuestionAlternative)<-[c:CHOSE]-(r:Respondent)
                MERGE (q)-[cb:CHOSEN_BY]->(r)
                SET cb.score = q.nrOfRespondents
               """)

In [ ]:
# Create graph on the new relationship CHOSEN_BY

gds.run_cypher("""
               CALL gds.graph.project(
                     'outkastRespondents',
                     ["QuestionAlternative", "Respondent"],
                     {
                            CHOSEN_BY: { 
                                properties: 'nrOfRespondents'
                            }
                     }
                ) YIELD graphName, nodeCount, relationshipCount
                AS graph
                RETURN graph
               """)

## Create a relation between Respondent and Survey: HAS_PARTICIPATED

In [ ]:


gds.run_cypher("""
                MATCH (su:Survey)-[:HAS_QUESTION]->(qu:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)<-[:CHOSE]-(re:Respondent)
                MERGE (re)-[:HAS_PARTICIPATED]->(su);
               """)

## Give each QuestionAlternative a percentage of the total repsondents who chose it.

In [ ]:
gds.run_cypher("""
                MATCH (su:Survey)-[:HAS_QUESTION]->(qu:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)
                WITH su, qa, qa.nrOfRespondents as nrOfRespondents
                SET qa.percentageOfRespondents = nrOfRespondents * 1.0 / su.totalParticipants
                RETURN qa.name as questionAlternativeName, qa.percentageOfRespondents as percentageOfRespondents, su.name as surveyName;
                """)

## Create a normality index.

In [ ]:
gds.run_cypher("""
                MATCH (re:Respondent)-[:CHOSE]->(qa:QuestionAlternative)
                WITH re, AVG(qa.percentageOfRespondents) as normalityIndex
                SET re.normalityIndex = normalityIndex
                RETURN re.id as respondentId, re.normalityIndex as normalityIndex
                ORDER BY normalityIndex;
               """)
